In [0]:
%run /Workspace/Users/vishal.krishnan@neudesic.com/Silver_to_Gold/utils

In [0]:
# # Define paths
# storage_account_name = "adcampaigndata"
# silver_container = "silver"
# gold_container = "gold"

# spark.conf.set(
#   f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net",
#   dbutils.secrets.get(scope="adls-creds", key="storage-key")
# )

In [0]:
# # Create Facts and Dims Tables

# from pyspark.sql.functions import *
# from pyspark.sql.types import *
# from delta.tables import DeltaTable
# from random import *
# from itertools import product

# spark.sql("CREATE SCHEMA IF NOT EXISTS gold")


# silver_base = f"abfss://{silver_container}@{storage_account_name}.dfs.core.windows.net/"
# gold_base = f"abfss://{gold_container}@{storage_account_name}.dfs.core.windows.net/"

# # Define paths to cleaned Silver data (update paths as per your setup)
# silver_youtube_path = silver_base + "Youtube_ads"
# silver_meta_path =  silver_base + "Meta_ads"
# silver_web_path = silver_base + "Web_analytics"

# # Load silver tables
# df_youtube = spark.read.format("delta").load(silver_youtube_path)
# df_meta = spark.read.format("delta").load(silver_meta_path)
# df_web = spark.read.format("delta").load(silver_web_path)


# # Upsert function
# def upsert_to_delta(df, table_name, keys):
#     if DeltaTable.isDeltaTable(spark, f"/mnt/gold/{table_name}"):
#         delta_table = DeltaTable.forPath(spark, f"/mnt/gold/{table_name}")
#         update_expr = {col: f"updates.{col}" for col in df.columns}
#         delta_table.alias("target").merge(
#             df.alias("updates"),
#             " AND ".join([f"target.{k} = updates.{k}" for k in keys])
#         ).whenMatchedUpdate(set=update_expr) \
#          .whenNotMatchedInsertAll() \
#          .execute()

#         if zorder_cols:
#             spark.sql(f"OPTIMIZE gold.{table_name} ZORDER BY ({', '.join(zorder_cols)})")
#     else:
#         df.write.format("delta").mode("overwrite").save(gold_base + f"/{table_name}")
#     table_path = gold_base + f"/{table_name}"
#     # Register or refresh the table in Hive metastore
#     spark.sql(f"""
#         CREATE TABLE IF NOT EXISTS gold.{table_name}
#         USING DELTA
#         LOCATION '{table_path}'
#     """)
#         if zorder_cols:
#                 spark.sql(f"OPTIMIZE gold.{table_name} ZORDER BY ({', '.join(zorder_cols)})")


# dim_campaign

# Select required columns from each source table, rename to match final schema
web_campaigns = df_web.select("campaign_id", "campaign_name", "campaign_start_date", "campaign_end_date") \
    .withColumnRenamed("campaign_start_date", "start_date") \
    .withColumnRenamed("campaign_end_date", "end_date")

meta_campaigns = df_meta.select("campaign_id", "campaign_name", "campaign_start_date", "campaign_end_date") \
    .withColumnRenamed("campaign_start_date", "start_date") \
    .withColumnRenamed("campaign_end_date", "end_date")

youtube_campaigns = df_youtube.select("campaign_id", "campaign_name", "campaign_start_date", "campaign_end_date") \
    .withColumnRenamed("campaign_start_date", "start_date") \
    .withColumnRenamed("campaign_end_date", "end_date")

# Union all sources and deduplicate
all_campaigns = web_campaigns.unionByName(meta_campaigns).unionByName(youtube_campaigns).distinct()

# Cast and enrich for final schema
campaign_df = all_campaigns.select(
    col("campaign_id"),
    col("campaign_name"),
    col("start_date").cast("date").alias("start_date"),
    col("end_date").cast("date").alias("end_date"),
    lit("Conversion").alias("objective"),
    lit(75000.00).cast(DecimalType(12, 2)).alias("budget"),
    lit("video").alias("creative_type")
).dropna(subset=["campaign_id", "campaign_name", "start_date", "end_date"])
# Upsert into Delta
upsert_to_delta(campaign_df, "dim_campaign", ["campaign_id"], zorder_cols=["campaign_id"])




# dim_platform
platform_df = spark.createDataFrame([
    ("101", "Facebook", True),
    ("102", "Instagram", True),
    ("103", "YouTube", True),
], ["platform_id", "platform_name", "is_paid_channel"])

upsert_to_delta(platform_df, "dim_platform", ["platform_id"], zorder_cols=["platform_id"])

# dim_date
from datetime import datetime, timedelta
start_date = datetime(2020, 1, 1)
end_date = datetime(2024, 12, 1)
date_rows = []
current = start_date
while current <= end_date:
    date_rows.append((current.strftime("%Y-%m-%d"), current.day, current.month, current.strftime("%b"),
                      f"Q{((current.month - 1) // 3) + 1}", current.year, current.strftime("%A")))
    current += timedelta(days=1)
dim_date_df = spark.createDataFrame(date_rows, ["date_key", "day", "month", "month_name", "quarter", "year", "day_of_week"])
dim_date_df = dim_date_df.withColumn("date_key", col("date_key").cast("date"))

upsert_to_delta(dim_date_df, "dim_date", ["date_key"], zorder_cols=["date_key"])


# dim_user

# Create separate arrays for age group, gender, region, and interest
age_groups = ["18-24", "25-34", "35-44", "45-54", "55-64", "65+"]
genders = ["Male", "Female", "Other"]
regions = ["India", "USA", "UK", "Canada", "Australia"]
interests = ["Tech", "Fashion", "Fitness", "Finance", "Travel", "Health", "Gaming"]

# Helper function to map age group string to numeric range
def age_from_group(age_group):
    if age_group == "65+":
        return random.randint(65, 75)
    start, end = age_group.split("-")
    return random.randint(int(start), int(end))

# Sample rows from df_web to create users
web_sample = df_web.select("session_id").withColumn("user_id", concat(lit("USER"), monotonically_increasing_id()))

# Randomly assign segment to each user and derive details from it
segment_sample = dim_user_segment.sample(True, 0.1).limit(web_sample.count())  # match count approximately
segment_sample = segment_sample.withColumn("row_id", monotonically_increasing_id())
web_sample = web_sample.withColumn("row_id", monotonically_increasing_id())

user_df = web_sample.select(
    col("user_id"),
    lit(choice(genders)).alias("gender"),
    lit(choice(age_groups)).alias("age_group"),
    lit(choice(interests)).alias("interest_category"),
    expr("""
        CASE
            WHEN '""" + choice(age_groups) + """' = '18-24' THEN CAST(rand() * 7 + 18 AS INT)
            WHEN '""" + choice(age_groups) + """' = '25-34' THEN CAST(rand() * 10 + 25 AS INT)
            WHEN '""" + choice(age_groups) + """' = '35-44' THEN CAST(rand() * 10 + 35 AS INT)
            WHEN '""" + choice(age_groups) + """' = '45-54' THEN CAST(rand() * 10 + 45 AS INT)
            WHEN '""" + choice(age_groups) + """' = '55-64' THEN CAST(rand() * 10 + 55 AS INT)
            WHEN '""" + choice(age_groups) + """' = '65+'   THEN CAST(rand() * 10 + 65 AS INT)
        END
    """).alias("age"),
    lit(choice(regions)).alias("location"),
    lit(choice(["Mobile", "Desktop"])).alias("device_type")
)

upsert_to_delta(user_df, "dim_user", ["user_id"], zorder_cols=["user_id"])





# FACT TABLES

# fact_engagement_metrics
engagement_df = df_youtube.select("campaign_id", "day", "impressions", "reach", "clicks", "link_clicks", "conversions", "likes", "shares", "comments") \
    .withColumn("platform_id", lit("youtube")) \
    .withColumnRenamed("day", "engagement_date")

meta_engage = df_meta.select("campaign_id", "date", "impressions", "reach", "clicks", "link_clicks", "likes", "comments", "shares", "conversions", "ad_spend", "platform_name") \
    .withColumn("platform_id", col("platform_name")) \
    .withColumnRenamed("date", "engagement_date")

fact_engagement = engagement_df.unionByName(meta_engage.select("campaign_id", "engagement_date", "impressions", "reach", "clicks", "link_clicks", "likes", "shares", "comments", "conversions", "platform_id"))
fact_engagement = fact_engagement.join(df_web.groupBy("campaign_id", to_date("session_start").alias("engagement_date")) \
    .agg(
        count("session_id").alias("sessions"),
        sum(when(col("bounce") == True, 1).otherwise(0)).alias("bounces"),
        countDistinct("session_id").alias("unique_users"),
        sum("revenue").alias("total_ad_spend")
    ), on=["campaign_id", "engagement_date"], how="left")

fact_engagement = fact_engagement.fillna(0)


upsert_to_delta(fact_engagement, "fact_engagement_metrics", ["campaign_id", "platform_id", "engagement_date"],
                zorder_cols=["campaign_id", "platform_id", "engagement_date"])



# fact_web_analytics
df_web = df_web.drop("campaign_start_date", "campaign_end_date")
fact_web = df_web.select(
    "*",
    (unix_timestamp("session_end") - unix_timestamp("session_start")).alias("duration_seconds"),
    col("bounce").alias("is_bounce"),
    lit("web").alias("platform_id"),
    concat(lit("USER"), monotonically_increasing_id()).alias("user_id")
)

upsert_to_delta(fact_web, "fact_web_analytics", ["session_id"], zorder_cols=["session_id"])


# fact_campaign_cost
meta_cost = df_meta.select("campaign_id", "date", "platform_name", "ad_spend", "clicks", "conversions", "likes", "comments", "shares") \
    .withColumn("total_engagement", col("likes") + col("comments") + col("shares")) \
    .withColumn("platform_id", col("platform_name")) \
    .withColumnRenamed("date", "cost_date")

fact_cost = meta_cost.select("campaign_id", "platform_id", "cost_date", "ad_spend", "clicks", "conversions", "total_engagement")

upsert_to_delta(fact_cost, "fact_campaign_cost", ["campaign_id", "platform_id", "cost_date"],
                zorder_cols=["campaign_id", "platform_id", "cost_date"])



# fact_audience_growth
audience_growth = df_web \
    .withColumn("platform_id", lit("web")) \
    .withColumn("date", to_date("session_start")) \
    .withColumn("user_id", concat(lit("USER"), monotonically_increasing_id())) \
    .groupBy("campaign_id", "platform_id", "date") \
    .agg(countDistinct("user_id").alias("engaged_users"))

upsert_to_delta(audience_growth, "fact_audience_growth", ["segment_id", "campaign_id", "platform_id", "date"],
                zorder_cols=["segment_id", "campaign_id", "platform_id", "date"])



# fact_ad_frequency
ad_frequency = df_web.select(
    concat(lit("USER"), monotonically_increasing_id()).alias("user_id"),
    col("campaign_id"),
    lit("web").alias("platform_id"),
    col("signed_up").alias("engagement"),
    lit(1).alias("impressions"),
    col("session_id")
).groupBy("user_id", "campaign_id", "platform_id", "engagement") \
 .agg(count("session_id").alias("impressions"))
 
upsert_to_delta(ad_frequency, "fact_ad_frequency", ["user_id", "campaign_id", "platform_id"],
                zorder_cols=["user_id", "campaign_id", "platform_id"])


# Read all the tables

# List of dimension and fact table names
dim_tables = [
    "dim_campaign",
    "dim_platform",
    "dim_date",
    "dim_user"
]

fact_tables = [
    "fact_engagement_metrics",
    "fact_web_analytics",
    "fact_campaign_cost",
    "fact_audience_growth",
    "fact_ad_frequency"
]

# Read all dim tables into a dictionary
dim_dfs = {table: spark.read.format("delta").load(gold_base + table) for table in dim_tables}

# Read all fact tables into a dictionary
fact_dfs = {table: spark.read.format("delta").load(gold_base + table) for table in fact_tables}

for i in dim_dfs:
    print(i)
    display(dim_dfs[i])
for i in fact_dfs:
    print(i)
    display(fact_dfs[i])


In [0]:
%sql
select * from  gold.fact_web_analytics

In [0]:
%sql
describe table gold.dim_campaign

In [0]:
%sql
DROP TABLE IF EXISTS gold.dim_campaign;
DROP TABLE IF EXISTS gold.dim_platform;
DROP TABLE IF EXISTS gold.dim_date;
DROP TABLE IF EXISTS gold.dim_user;
DROP TABLE IF EXISTS gold.fact_engagement_metrics;
DROP TABLE IF EXISTS gold.fact_web_analytics;
DROP TABLE IF EXISTS gold.fact_campaign_cost;
DROP TABLE IF EXISTS gold.fact_audience_growth;
DROP TABLE IF EXISTS gold.fact_ad_frequency;

In [0]:
dim_tables = [
    "dim_campaign",
    "dim_platform",
    "dim_date",
    "dim_user"
]

fact_tables = [
    "fact_engagement_metrics",
    "fact_web_analytics",
    "fact_campaign_cost",
    "fact_audience_growth",
    "fact_ad_frequency"
]